In [13]:
from importlib import reload

import ethernet
import func

class PFI(object):
    def __init__(self, fpgaHost='localhost'):
        self.fpgaHost = fpgaHost
        self.fpga = None
        
    def connect(self):
        ethernet.sock.connect(self.fpga, 4001)
        
    def disconnect(self):
        ethernet.sock.close()
        ethernet.sock = None
        
    def loadModel(self, filename=None):
        import ics.cobraOps.CobrasCalibrationProduct as cobraModel
        reload(cobraModel)

        if filename is None:
            filename = "/Users/cloomis/Sumire/PFS/git/ics_cobraOps/python/ics/cobraOps/usedXMLFile.xml"
        
        self.calibModel = cobraModel.CobrasCalibrationProduct(filename)
        
    def setFreq(self, cobras):
        thetaPeriods = []
        phiPeriods = []
        for c in cobras:
            print(f'setting {c.cobra}, {calibs.motorFreq1[c.cobra]}, {calibs.motorFreq2[c.cobra]}')
            thetaPeriods.append(func.get_per(calibs.motorFreq1[c.cobra]))
            phiPeriods.append(func.get_per(calibs.motorFreq2[c.cobra]))
        func.setFreq(cobras, thetaPeriods, phiPeriods)
        
def allocateCobraRange(boards, cobraRange=None):
    if cobraRange is None:
        cobraRange = 0,57
    cobras = []
    for b in boards:
        for c in range(cobraRange[0], cobraRange[1]):
            cobras.append(func.Cobra(b, c))
        
    return cobras

def allocateCobraList(cobraIds):
    cobras = []
    for bc in cobraIds:
        b, c = bc
        cobras.append(func.Cobra(b, c))
        
    return cobras

def testRuns(cobras):
    for c in cobras:
        c.p = func.RunParams(pu=(70,70),
                             st=(10,10),
                             en=(True, True),
                             dir=('cw', 'cw'))

In [4]:
if "allCobras" != "allCobras":
    cobras = allocateCobraRange(range(42))
else:
    cobras = allocateCobraList([(1,1)])

In [5]:
testRuns(cobras)
len(cobras)

1

In [6]:
func.RUN(cobras)

False

In [6]:
%pdb on
func.run(cobras, [5], [5], [0.07], [0.03], dirs=('cw', 'cw'))
%pdb off

Automatic pdb calling has been turned ON


TypeError: unsupported operand type(s) for >>: 'float' and 'int'

> /Users/cloomis/Sumire/PFS/git/pfs_gui_revF/func.py(105)toList()
    103         c = 0x0000 | (self.dir[0]=='ccw') | ((self.dir[1]=='ccw')<<1) | \
    104                 (self.en[0]<<2) | (self.en[1]<<3) | (board<<4) | (cnum%30<<11)
--> 105         p = [c>>8, c%256, self.pulses[0]>>8, self.pulses[0]%256, self.steps[0]>>8, \
    106                 self.steps[0]%256, self.sleeps[0]>>8, self.sleeps[0]%256, \
    107                 self.pulses[1]>>8, self.pulses[1]%256, self.steps[1]>>8, \

ipdb> c


In [7]:
func.RUN(cobras, timeout=5)

TypeError: unsupported operand type(s) for >>: 'float' and 'int'

> /Volumes/ssdata/git/pfs_gui_revF/func.py(105)toList()
    103         c = 0x0000 | (self.dir[0]=='ccw') | ((self.dir[1]=='ccw')<<1) | \
    104                 (self.en[0]<<2) | (self.en[1]<<3) | (board<<4) | (cnum%30<<11)
--> 105         p = [c>>8, c%256, self.pulses[0]>>8, self.pulses[0]%256, self.steps[0]>>8, \
    106                 self.steps[0]%256, self.sleeps[0]>>8, self.sleeps[0]%256, \
    107                 self.pulses[1]>>8, self.pulses[1]%256, self.steps[1]>>8, \

ipdb> p c
2076
ipdb> p self.pulses
(0.01, 0.03)
ipdb> p self.steps
(1, 3)
ipdb> c


In [7]:
for i in range(10):
    func.RST()

In [26]:
func.EXIT()

True

In [10]:
%pdb on
setFreq(cobras)
%pdb off

Automatic pdb calling has been turned ON
setting 1, 63241.0, 104575.0


TypeError: unsupported operand type(s) for >>: 'numpy.float64' and 'int'

> /Users/cloomis/Sumire/PFS/git/pfs_gui_revF/func.py(124)toList()
    122         c = 0x0000 | (self.en[0]) | (self.en[1]<<1) | \
    123                 (board<<4) | (cnum%30<<11)
--> 124         p = [c>>8, c%256, self.m0Per>>8, self.m0Per%256, \
    125                 self.m1Per>>8, self.m1Per%256 ]
    126         return p

ipdb> c


In [19]:
print(calibs.motorFreq1, calibs.motorFreq2)
print(calibs.motorOntimeFwd1, calibs.motorOntimeFwd2)

[ 63241.  63241.  63745.  63492.  63241.  63492.  62500.  63492.  64000.
  63492.  62992.  63241.  62992.  63492.  64000.  63492.  63492.  62992.
  63241.  64000.  63745.  62992.  63492.  64000.  63241.  62500.  63241.
  63241.  63492.  63492.  63745.  63745.  63745.  63241.  63241.  64000.
  62745.  64000.  62745.  62992.  63492.  62500.  63241.  62256.  64000.
  63492.  64000.  62992.  63492.  64516.  64000.  64257.  64516.  63745.
  64000.  64777.  63492.] [ 108108.  104575.  103896.  104575.  105263.  105960.  105960.  105263.
  104575.  105960.  107382.  105263.  105960.  105263.  105960.  103896.
  105263.  106666.  105263.  105960.  105960.  104575.  104575.  105263.
  105960.  106666.  104575.  108108.  105960.  105263.  108843.  106666.
  105263.  105263.  106666.  105263.  105960.  105263.  104575.  106666.
  105960.  105263.  105960.  104575.  105960.  105263.  105960.  105263.
  106666.  106666.  105960.  105960.  104575.  104575.  103896.  107382.
  103896.]
[ 0.04572   0.

In [20]:
print(calibs.S1Nm[0])

[  70.03209804   62.74728531   56.84150693   57.90853668   57.06857741
   57.06857741   58.05328001   58.05328001   61.63222681   61.63222681
   61.63222681   69.17223887   76.50784205   76.50784205   77.79746726
  108.14058276  109.15375519  122.17056368   81.28062135   46.92816081
  118.288756    105.69583089   82.41003571  118.288756     73.87191431
  107.69414862  113.66147823  115.45492447  115.45492447  133.71466776
   97.19222462   80.79719903   85.51678267   95.06456468  167.15419975
  130.84248019  187.44142455  139.67022308   78.51347815   67.68316757
   67.50421901   49.18906363   51.51762332   81.73644537   73.56246679
   67.12785993   61.17559094   59.56714541   59.99700015   56.73401204
   59.70347275   57.34218952   64.28112278   69.29606745   52.83395462
   49.94381321   50.07720235   52.8262018    55.78022591   95.67852017
   74.02228893   68.79024707   68.02078413   72.07207207   70.29876977
   70.06480995   73.40646793   69.82698425   66.74577277   66.85857554
   44.